In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
import time, os

# ============================================================
# CONFIG
# ============================================================
IMG_SIZE = (300, 300)       # 🔥 Recommended for EfficientNet-B3
BATCH_SIZE = 32
EPOCHS = 20
LR = 1e-3
train_dir = r"./images"
class_file = r"./classes.txt"

# ✅ Use new weight/checkpoint names to avoid conflicts
best_model_path = "best_food_classifier_b3.pth"
checkpoint_path = "checkpoint_b3.pth"

# ============================================================
# LOAD CLASS NAMES
# ============================================================
with open(class_file, "r") as f:
    class_names = [line.strip() for line in f.readlines()]
num_classes = len(class_names)

# ============================================================
# DEVICE SETUP
# ============================================================
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔍 Using device: {device}")
if device.type == "cuda":
    print(f"✅ GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ Running on CPU — check CUDA setup if unexpected.")

# ============================================================
# MODEL CREATION
# ============================================================
def create_model(num_classes):
    model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.IMAGENET1K_V1)

    # Freeze feature extractor initially
    for param in model.features.parameters():
        param.requires_grad = False

    # Replace classifier head
    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.4),
        nn.Linear(in_features, num_classes)
    )
    return model

# ============================================================
# DATA AUGMENTATION & LOADERS
# ============================================================
transform = transforms.Compose([
    transforms.RandomResizedCrop(300, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.3, 0.3, 0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size],
                                          generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8, pin_memory=True)

# ============================================================
# TRAIN FUNCTION (with better unfreezing)
# ============================================================
def train_model(model, criterion, optimizer, scheduler, scaler, start_epoch=0, best_val_acc=0.0):
    total_blocks = len(model.features)
    print(f"🧩 Total EfficientNet feature blocks: {total_blocks}")

    # Progressive unfreezing schedule
    unfreeze_schedule = {
        5: int(0.75 * total_blocks),  # Unfreeze last 25%
        10: int(0.5 * total_blocks),  # Unfreeze last 50%
        15: 0                         # Unfreeze entire network
    }

    for epoch in range(start_epoch, EPOCHS):
        # ---------------------------------------
        # Progressive unfreezing
        # ---------------------------------------
        if epoch in unfreeze_schedule:
            unfreeze_from = unfreeze_schedule[epoch]
            print(f"\n🔓 Epoch {epoch+1}: Unfreezing from block {unfreeze_from} onwards...")
            for param in model.features[unfreeze_from:].parameters():
                param.requires_grad = True
            for g in optimizer.param_groups:
                g['lr'] *= 0.5
            print(f"✅ Adjusted LR to {g['lr']:.2e}")

        # ---------------------------------------
        # TRAIN LOOP
        # ---------------------------------------
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        start_time = time.time()

        for images, labels in train_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_dataset)
        train_acc = correct / total

        # ---------------------------------------
        # VALIDATION LOOP
        # ---------------------------------------
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_dataset)
        val_acc = val_correct / val_total
        scheduler.step(val_loss)
        epoch_time = time.time() - start_time

        # ---------------------------------------
        # LOGGING
        # ---------------------------------------
        print(f"\n📅 Epoch [{epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | "
              f"⏱ Time: {epoch_time:.1f}s")

        if device.type == "cuda":
            allocated = torch.cuda.memory_allocated(0) / 1024**2
            reserved = torch.cuda.memory_reserved(0) / 1024**2
            print(f"🧠 GPU Memory -> Allocated: {allocated:.1f} MB | Reserved: {reserved:.1f} MB")

        # Save checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_val_acc': best_val_acc,
        }, checkpoint_path)

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"💾 Saved new best B3 model (Val Acc: {best_val_acc:.4f})")

    print("\n✅ Training complete.")
    return model

# ============================================================
# TRAIN FROM SCRATCH (ignore any old checkpoints)
# ============================================================
print("🆕 Starting fresh EfficientNet-B3 training (no checkpoint resume).")
model = create_model(num_classes).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # 🔥 helps generalization
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)  # better regularization
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.3, verbose=True)
scaler = GradScaler()

# Train the model
model = train_model(model, criterion, optimizer, scheduler, scaler)

# Final save
torch.save(model.state_dict(), "final_food_classifier_b3.pth")
print("\n✅ Final EfficientNet-B3 model saved successfully!")


🔍 Using device: cuda
✅ GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU
🆕 Starting fresh EfficientNet-B3 training (no checkpoint resume).


C:\Users\sreep\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\sreep\AppData\Local\Temp\ipykernel_18004\22126182.py:193: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


🧩 Total EfficientNet feature blocks: 9


C:\Users\sreep\AppData\Local\Temp\ipykernel_18004\22126182.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\sreep\AppData\Local\Temp\ipykernel_18004\22126182.py:141: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():



📅 Epoch [1/25] Train Loss: 2.9502 | Train Acc: 0.4140 | Val Loss: nan | Val Acc: 0.5407 | ⏱ Time: 612.7s
🧠 GPU Memory -> Allocated: 68.1 MB | Reserved: 82.0 MB
💾 Saved new best B3 model (Val Acc: 0.5407)

📅 Epoch [2/25] Train Loss: 2.5774 | Train Acc: 0.4900 | Val Loss: nan | Val Acc: 0.5661 | ⏱ Time: 650.3s
🧠 GPU Memory -> Allocated: 68.1 MB | Reserved: 768.0 MB
💾 Saved new best B3 model (Val Acc: 0.5661)

📅 Epoch [3/25] Train Loss: 2.5452 | Train Acc: 0.5013 | Val Loss: nan | Val Acc: 0.5756 | ⏱ Time: 635.1s
🧠 GPU Memory -> Allocated: 68.1 MB | Reserved: 768.0 MB
💾 Saved new best B3 model (Val Acc: 0.5756)

📅 Epoch [4/25] Train Loss: 2.5346 | Train Acc: 0.5054 | Val Loss: nan | Val Acc: 0.5835 | ⏱ Time: 450.1s
🧠 GPU Memory -> Allocated: 68.1 MB | Reserved: 768.0 MB
💾 Saved new best B3 model (Val Acc: 0.5835)

📅 Epoch [5/25] Train Loss: 2.4698 | Train Acc: 0.5230 | Val Loss: nan | Val Acc: 0.5935 | ⏱ Time: 419.7s
🧠 GPU Memory -> Allocated: 68.1 MB | Reserved: 768.0 MB
💾 Saved new bes

KeyboardInterrupt: 